In [1]:
import pandas as pd
import re
import yfinance as yf
from tqdm.auto import tqdm
import json

/Users/derekkuang/cse158_assignment2/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [19]:
wsb = pd.read_csv("data/wsb.csv")
wsb

,register_index,post_id,comment_id,author,datetime,title,url,score,comments,text,author_post_karma,tag
0,14b78hkjoe86nf,14b78hk,joe86nf,scott_jr,2023-06-16 20:36:55,NaN,NaN,1.0,NaN,Watch til 1 10,32102.0,Meme
1,14b71m2post,14b71m2,NaN,merakibret,2023-06-16 20:24:01,I had my first ever big success with options t...,https://www.reddit.com/r/wallstreetbets/commen...,8.0,6.0,Entered an Iron Condor on ADBE yesterday at 45...,343.0,Gain
2,14b71m2joe6du9,14b71m2,joe6du9,VisualMod,2023-06-16 20:24:07,NaN,NaN,1.0,NaN,User Report Tota...,725083.0,Gain
3,14b71m2joe6een,14b71m2,joe6een,VisualMod,2023-06-16 20:24:13,NaN,NaN,2.0,NaN,That was a very wise move,725083.0,Gain
4,14b71m2joe7yy4,14b71m2,joe7yy4,DreamcatcherEgg,2023-06-16 20:35:23,NaN,NaN,2.0,NaN,All you have to do is repeat this same winning...,6088.0,Gain
...,...,...,...,...,...,...,...,...,...,...,...,...
3033535,1j96owemhd8ajs,1j96owe,mhd8ajs,jarail,2025-03-12 11:25:28,NaN,NaN,4.0,NaN,Hopefully he made the whole story up,NaN,Discussion
3033536,1j96owemhcapeb,1j96owe,mhcapeb,South_Age974,2025-03-12 05:46:48,NaN,NaN,2.0,NaN,holy f,NaN,Discussion
3033537,1j96owemhjt5np,1j96owe,mhjt5np,PickinLosers,2025-03-13 11:49:05,NaN,NaN,2.0,NaN,I like to call them PDFs public domain fries,NaN,Discussion
3033538,1j96owemhex8ls,1j96owe,mhex8ls,The_Whackest,2025-03-12 17:04:15,NaN,NaN,1.0,NaN,Singsongy Some make you laugh and others ma...,NaN,Discussion


In [20]:
wsb.info()
wsb.head()
wsb.shape

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3033540 entries, 0 to 3033539
Data columns (total 12 columns):
 #   Column             Dtype  
---  ------             -----  
 0   register_index     object 
 1   post_id            object 
 2   comment_id         object 
 3   author             object 
 4   datetime           object 
 5   title              object 
 6   url                object 
 7   score              float64
 8   comments           float64
 9   text               object 
 10  author_post_karma  float64
 11  tag                object 
dtypes: float64(3), object(9)
memory usage: 277.7+ MB


(3033540, 12)

In [21]:
# Clean and normalize dataset

# drop uneccessary columns
wsb = wsb.drop(columns=["register_index"])

# convert datetime column to datetime type and normalize to date only
wsb["datetime"] = pd.to_datetime(wsb["datetime"], errors="coerce").dt.normalize()

# convert numeric columns to int type and fill na with 0
numeric_cols = ["score", "comments", "author_post_karma"]
for col in numeric_cols:
    wsb[col] = pd.to_numeric(wsb[col], errors="coerce")
    wsb[col] = wsb[col].fillna(0).astype(int)

# clean and normalize tag column
wsb["tag"] = (
    wsb["tag"].fillna("unknown").str.strip().str.lower().str.replace(r"\s+", "_", regex=True)
)
wsb["tag"] = wsb["tag"].astype("category")

# identify post or comment
wsb["is_post"] = wsb["comment_id"].isna()

wsb.info()
wsb.head()
wsb.shape

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3033540 entries, 0 to 3033539
Data columns (total 12 columns):
 #   Column             Dtype         
---  ------             -----         
 0   post_id            object        
 1   comment_id         object        
 2   author             object        
 3   datetime           datetime64[ns]
 4   title              object        
 5   url                object        
 6   score              int64         
 7   comments           int64         
 8   text               object        
 9   author_post_karma  int64         
 10  tag                category      
 11  is_post            bool          
dtypes: bool(1), category(1), datetime64[ns](1), int64(3), object(6)
memory usage: 237.2+ MB


(3033540, 12)

In [22]:
# basic filtering / denoising

# remove obvious bot/moderator authors
bot_users = ["VisualMod", "AutoModerator"]
wsb = wsb[~wsb["author"].isin(bot_users)]

# drop rows with deleted/empty content
garbage_tokens = {"", "[deleted]", "[removed]"}
title_clean = wsb["title"].fillna("").str.strip()
text_clean = wsb["text"].fillna("").str.strip()

title_garbage = title_clean.isin(garbage_tokens)
text_garbage = text_clean.isin(garbage_tokens)

post_mask = wsb["is_post"]
comment_mask = ~wsb["is_post"]

drop_mask = (post_mask & title_garbage & text_garbage) | (comment_mask & text_garbage)
wsb = wsb[~drop_mask]

wsb.shape

(2956091, 12)

In [23]:
# light initial text cleaning
# build canonical text field for NLP
title_component = wsb["title"].fillna("").astype(str)
text_component = wsb["text"].fillna("").astype(str)

post_raw_text = (title_component + "\n\n" + text_component).str.strip()
comment_raw_text = text_component.str.strip()

wsb["raw_text"] = post_raw_text.where(wsb["is_post"], comment_raw_text)
wsb = wsb[wsb["raw_text"] != ""]

# light text normalization for downstream models
import re
url_pattern = re.compile(r"http\S+")

def clean_text_value(s: str) -> str:
    s = str(s)
    s = url_pattern.sub("", s)
    s = s.replace("\n", " ")
    s = re.sub(r"\s+", " ", s)
    return s.strip()

wsb["clean_text"] = wsb["raw_text"].map(clean_text_value).str.lower()


wsb.shape

(2956091, 14)

In [11]:
wsb = pd.read_csv('data/wsb_cleaned.csv')

In [12]:
# Simple regexes
cashtag_pattern = re.compile(r'\$[A-Za-z]{1,5}')
upper_pattern   = re.compile(r'\b[A-Z]{2,5}\b')  # 2–5 uppercase letters

def extract_candidate_tokens(text):
    if not isinstance(text, str):
        return []

    cands = set()

    # $TSLA, $GME
    for m in cashtag_pattern.findall(text):
        cands.add(m[1:].upper())  # strip '$'

    # TSLA, GME, NVDA (bare tickers)
    for m in upper_pattern.findall(text):
        cands.add(m.upper())

    return list(cands)

wsb['candidate_tokens'] = wsb['raw_text'].apply(extract_candidate_tokens)

tokens_exploded = wsb.explode('candidate_tokens')
tokens_exploded = tokens_exploded.dropna(subset=['candidate_tokens'])

token_counts = (
    tokens_exploded
    .groupby('candidate_tokens')
    .size()
    .reset_index(name='count')
    .sort_values('count', ascending=False)
)

token_counts.head(30)





,candidate_tokens,count
11132,NVDA,49561
381,AI,40492
16203,US,29507
11519,OP,26281
14563,SPY,25820
17298,WSB,19541
15743,TSLA,17708
578,AMD,13938
2545,CEO,10445
3621,DD,9184


In [ ]:
# -------------------------
# 1. Filter candidate tokens
# -------------------------
min_count = 500  # as you used; adjust if needed
freq_candidates = (
    token_counts[token_counts['count'] >= min_count]['candidate_tokens']
    .astype(str)
    .tolist()
)

print(f"Number of freq_candidates (count >= {min_count}): {len(freq_candidates)}")

# -------------------------------------------
# 2. Build valid_tickers via yfinance (whitelist)
# -------------------------------------------
start_date = '2023-06-08'
end_date   = '2025-04-02'

valid_tickers_all = []

for sym in tqdm(freq_candidates, desc="Checking yfinance symbols"):
    try:
        data = yf.download(sym, start=start_date, end=end_date, progress=False)
        if not data.empty:
            valid_tickers_all.append(sym)
    except Exception:
        # Skip symbols that cause errors
        continue

valid_tickers_all = sorted(set(valid_tickers_all))
print(f"Number of symbols with real price data: {len(valid_tickers_all)}")

# --------------------------------------------------
# 3. Take the top 150 valid tickers by WSB frequency
# --------------------------------------------------
vc = (
    token_counts[token_counts['candidate_tokens'].isin(valid_tickers_all)]
    .copy()
    .sort_values('count', ascending=False)
)

top_200 = vc['candidate_tokens'].head(200).tolist()
print(f"Initial top_200 tickers (before manual removal): {len(top_200)}")

# You can inspect this manually if you want:
# print(top_150)

# --------------------------------------------------
# 4. Remove obviously ambiguous / non-stock symbols
#    (we prune the top_150 list instead of defining a
#     separate manual_keep list)
# --------------------------------------------------
manual_drop = {
    # Common words / pronouns / fillers
    "IT", "AM", "ARE", "FOR", "ALL", "GO", "NOW", "OR", "JUST", "YOU",
    "MORE", "TIME", "LOT", "WAY", "BACK", "DAY",

    # Macro / econ / generic finance
    "US", "USA", "USD", "CPI", "PPI", "EPS", "IRS",
    "IPO", "ITM", "PM", "EOD",

    # Crypto / FX / non-equity focus
    "BTC", "ETH", "SOL", "DOGE", "ADA", "XRP", "USDT", "USDC",

    # Slang / memes / platform lingo
    "YOLO", "WTF", "IMO", "MOON", "GL",

    # Generic financial/technical terms (not single equities)
    "IV", "ETF", "RSI",

    # Regions / places
    "EU", "UK", "NYC", "CA", "DC",

    # Ambiguous tickers that are almost always normal words or other concepts
    "AI", "OP", "DD", "EV", "LINK", "TV", "ON", "UP", "VR", "PC", "SO",
    "IP", "CC", "IQ", "OPEN", "BE", "CAN", "MS", "OS", "PT", "PDT",
    "COST", "DEI", "OUT", "HE", "UI", "PR", "AM", "AGI", "ICE",

    # Misc abbreviations that are usually not equity tickers in WSB text
    "EPS", "CFO", "AA", "GPT", "HERE", "LOVE", "ANY"
}


filtered_candidates = [t for t in top_200 if t not in manual_drop]
print(f"After manual_drop filtering: {len(filtered_candidates)} symbols")

# ---------------------------------------------------------
# 5. From these filtered candidates, keep the TOP 100 by
#    WSB frequency (if we have >= 100; else keep all)
# ---------------------------------------------------------
vc_filtered = vc[vc['candidate_tokens'].isin(filtered_candidates)].copy()
vc_filtered = vc_filtered.sort_values('count', ascending=False)

top_n = 100
final_tickers = vc_filtered['candidate_tokens'].head(top_n).tolist()
print(f"Final ticker universe size (up to {top_n}): {len(final_tickers)}")
print("Preview final_tickers:", final_tickers[:20])

# Save final universe to JSON for verification
valid_tickers_path = 'data/valid_tickers_top100.json'
with open(valid_tickers_path, 'w') as f:
    json.dump(final_tickers, f, indent=2)

print(f"Saved final tickers to {valid_tickers_path}")

valid_tickers_set = set(final_tickers)

# ------------------------------------------------
# 6. Re-extract tickers from WSB text using whitelist
# ------------------------------------------------
cashtag_pattern = re.compile(r'\$[A-Za-z]{1,5}')
upper_pattern   = re.compile(r'\b[A-Z]{2,5}\b')

def extract_tickers(text):
    if not isinstance(text, str):
        return []

    cands = set()

    # $TSLA/$GME style
    for m in cashtag_pattern.findall(text):
        cands.add(m[1:].upper())

    # TSLA/GME style (bare)
    for m in upper_pattern.findall(text):
        cands.add(m.upper())

    # keep only in the curated top-100 universe
    return [t for t in cands if t in valid_tickers_set]

wsb['tickers'] = wsb['raw_text'].apply(extract_tickers)

# Keep only rows that mention at least one of our final tickers
wsb_with_ticker = wsb[wsb['tickers'].str.len() > 0].copy()

# One row per (WSB row, ticker)
wsb_exploded = (
    wsb_with_ticker
    .explode('tickers')
    .rename(columns={'tickers': 'ticker'})
)

print("Exploded WSB shape:", wsb_exploded.shape)
print(wsb_exploded[['datetime', 'ticker']].head())

# Optional: sanity check ticker counts
print("\nTop 20 tickers after final filter:")
print(wsb_exploded['ticker'].value_counts().head(20))

#wsb_exploded.to_csv('data/wsb_exploded.csv', index=False)

Number of freq_candidates (count >= 500): 303


Checking yfinance symbols:   0%|          | 0/303 [00:00<?, ?it/s]/var/folders/_m/g3l0_2ld7859l89b672mt3k40000gn/T/ipykernel_51073/3529361275.py:23: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(sym, start=start_date, end=end_date, progress=False)
Checking yfinance symbols:   0%|          | 1/303 [00:00<01:03,  4.79it/s]/var/folders/_m/g3l0_2ld7859l89b672mt3k40000gn/T/ipykernel_51073/3529361275.py:23: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(sym, start=start_date, end=end_date, progress=False)
Checking yfinance symbols:   1%|          | 2/303 [00:00<00:44,  6.81it/s]/var/folders/_m/g3l0_2ld7859l89b672mt3k40000gn/T/ipykernel_51073/3529361275.py:23: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(sym, start=start_date, end=end_date, progress=False)

1 Failed download:
['US']: YFPricesMissingError('possibly delisted; no price dat

Number of symbols with real price data: 161
Initial top_200 tickers (before manual removal): 161
After manual_drop filtering: 87 symbols
Final ticker universe size (up to 100): 87
Preview final_tickers: ['NVDA', 'SPY', 'TSLA', 'AMD', 'SMCI', 'QQQ', 'AAPL', 'MSFT', 'TSM', 'MSTR', 'AMC', 'PLTR', 'INTC', 'DJT', 'RH', 'META', 'ASTS', 'ARM', 'AMZN', 'MARA']
Saved final tickers to data/valid_tickers_top100.json
Exploded WSB shape: (245459, 17)
      datetime ticker
1   2023-06-16   ADBE
13  2023-06-16    SPY
26  2023-06-16     TD
30  2023-06-16    IWM
32  2023-06-16    SPY

Top 20 tickers after final filter:
ticker
NVDA    49561
SPY     25820
TSLA    17708
AMD     13938
SMCI     5930
QQQ      5655
AAPL     5422
MSFT     4839
TSM      4827
MSTR     4191
AMC      4121
PLTR     3992
INTC     3688
DJT      3673
RH       3511
META     3479
ASTS     3062
ARM      2828
AMZN     2714
MARA     2663
Name: count, dtype: int64


In [14]:
wsb_exploded.shape

(245459, 17)

In [15]:
# ---------------------------------------------
# 1. Load your final ticker universe from JSON
# ---------------------------------------------
tickers_path = 'data/valid_tickers_top100.json'  # adjust if different

with open(tickers_path, 'r') as f:
    final_tickers = json.load(f)

print(f"# tickers: {len(final_tickers)}")
print("Preview:", final_tickers[:10])

# ---------------------------------------------
# 2. Define date range
# ---------------------------------------------
start_date = '2023-06-01'
end_date   = '2025-04-02'

# ---------------------------------------------
# 3. Single multi-ticker download (WIDE, MultiIndex)
# ---------------------------------------------
raw = yf.download(
    final_tickers,
    start=start_date,
    end=end_date,
    auto_adjust=False,   # keep raw OHLC
    progress=False
)

# raw is wide with MultiIndex columns: (PriceField, Ticker)
print("Raw shape:", raw.shape)
print("Column level names:", raw.columns.names)
print(raw.head(3))

# ---------------------------------------------
# 4. Reshape to LONG: one row per (date, ticker)
# ---------------------------------------------
# Ensure column levels have names (should already be ['Price', 'Ticker'])
if raw.columns.names is None or len(raw.columns.names) != 2:
    # yfinance usually sets this, but just in case:
    raw.columns.names = ['Price', 'Ticker']

# Move 'Ticker' from column level to a column via stack
# After this:
#   index: Date
#   column: Price fields (Open, High, Low, Close, Adj Close, Volume)
#   plus a 'Ticker' column from stacking
prices_long = (
    raw
    .stack(level='Ticker')          # stack over ticker level → ticker becomes index level
    .reset_index()                  # turn index back into columns
    .rename(columns={'Date': 'datetime', 'Ticker': 'ticker'})
)

# Now columns should be: ['datetime','ticker','Open','High','Low','Close','Adj Close','Volume']
print("After stack → long shape:", prices_long.shape)
print(prices_long.head(3))
print("Columns after stack:", prices_long.columns.tolist())

# ---------------------------------------------
# 5. Clean column names & add 'date'
# ---------------------------------------------
prices_long = prices_long.rename(
    columns={
        'Open': 'open',
        'High': 'high',
        'Low': 'low',
        'Close': 'close',
        'Adj Close': 'adj_close',
        'Volume': 'volume'
    }
)

prices_long['datetime'] = pd.to_datetime(prices_long['datetime'])

# Sort for sanity
prices_long = prices_long.sort_values(['ticker', 'datetime']).reset_index(drop=True)

print("Final LONG price data shape:", prices_long.shape)
print(prices_long.head(5))
print("Final columns:", prices_long.columns.tolist())

out_path = 'data/prices_daily.csv'
prices_long.to_csv(out_path, index=False)

# tickers: 87
Preview: ['NVDA', 'SPY', 'TSLA', 'AMD', 'SMCI', 'QQQ', 'AAPL', 'MSFT', 'TSM', 'MSTR']
Raw shape: (460, 522)
Column level names: ['Price', 'Ticker']
Price        Adj Close                                                       \
Ticker            AAPL  ACHR        ADBE        AMC         AMD        AMZN   
Date                                                                          
2023-06-01  177.930115  2.99  426.750000  45.500000  119.470001  122.769997   
2023-06-02  178.779846  2.96  436.369995  45.500000  117.860001  124.250000   
2023-06-05  177.426270  3.00  434.179993  46.299999  117.930000  125.300003   

Price                                    ...    Volume                       \
Ticker         AR ARM        ASML  ASTS  ...      TLRY       TLT       TQQQ   
Date                                     ...                                  
2023-06-01  20.16 NaN  710.749268  5.60  ...  26101500  21040400  216759200   
2023-06-02  21.32 NaN  708.676086  5.62  ...  2

/var/folders/_m/g3l0_2ld7859l89b672mt3k40000gn/T/ipykernel_51073/2961499495.py:49: FutureWarning: The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.
  .stack(level='Ticker')          # stack over ticker level → ticker becomes index level


In [16]:
prices_long.shape

(39582, 8)